In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
f=open(r'templeSparseRing\templeSR_par.txt','r')
assert f is not None
n= int(f.readline())

#reading the information of the 2nd image 
l=f.readline().split()
im1_fn=l[0]

k1=np.array([float(i) for i in l[1:10]]).reshape((3,3))
R1=np.array([float(i) for i in l[10:19]]).reshape((3,3))
t1=np.array([float(i) for i in l[19:22]]).reshape((3,1))

#reading the information of the 2nd image 

l=f.readline().split()
im2_fn=l[0]

k2=np.array([float(i) for i in l[1:10]]).reshape((3,3))
R2=np.array([float(i) for i in l[10:19]]).reshape((3,3))
t2=np.array([float(i) for i in l[19:22]]).reshape((3,1))

#read the two images and show
im1= cv.imread(r'templeSparseRing/' + im1_fn,cv.IMREAD_COLOR)
im2= cv.imread(r'templeSparseRing/' + im2_fn,cv.IMREAD_COLOR)
assert im1 is not None
assert im2 is not None



In [12]:
sift = cv.xfeatures2d.SIFT_create()
kp1, decs1 = sift.detectAndCompute(im1, None)
kp2, decs2 = sift.detectAndCompute(im2, None)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm =FLANN_INDEX_KDTREE, trees = 5 )
search_params = dict(check=100)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(decs1, decs2, k=2)

good = []
pts1 = []
pts2 = []
for i, (m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        good.append(m)
        pts1.append(kp1[m.queryIdx].pt)
        pts2.append(kp2[m.trainIdx].pt)

pts1 = np.array(pts1)
pts2 = np.array(pts2)

F,mask = cv.findFundamentalMat(pts1, pts2, cv.FM_RANSAC)

E = k2.T @ F @ k1
retval, R, t, mask = cv.recoverPose(E, pts1, pts2, k1)

R_t_1 = np.concatenate((R1, t1), axis =1) # 3 x 4
R2_ = R1 @ R
t2_ = R1 @ t
R_t_2 = np.concatenate(R2_, )
print(F)

[[-2.92272774e-06 -1.07825108e-05 -5.69762921e-02]
 [ 2.14158681e-05 -1.12610694e-07 -1.22955856e-02]
 [ 5.69647752e-02  5.90541351e-03  1.00000000e+00]]
